Test and evaluate the LLM

In [1]:
%pip install torch
%pip install peft
%pip install transformers

%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [2]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

peft_model_id = "./models/codeLlama-7b-instruct-hf-text-to-sql" #"./models/codeLlama-7b-text-to-sql"
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
# model = AutoModelForCausalLM.from_pretrained(
#   peft_model_id,
#   device_map="auto",
#   torch_dtype=torch.float16
# )
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


/root/dev/genai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.53s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM',

In [6]:

from datasets import load_dataset
from random import randint

# Load our test dataset

eval_dataset = load_dataset("json", data_files="retrain/test_dataset.json", split="train")
# rand_idx = randint(0, len(eval_dataset))


for rand_idx in range(len(eval_dataset)):
    # Test on sample
    print(f"Test:\n{eval_dataset[rand_idx]['messages'][:2]}")

    prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    
    print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
    print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
    print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Generating train split: 100 examples [00:00, 13163.97 examples/s]


Test:
[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_83 (birth_place VARCHAR, monarchs_served VARCHAR, entered_office VARCHAR)', 'role': 'system'}, {'content': 'What is the birth place of the prime minister who served George V and entered office on 23 October 1922?', 'role': 'user'}]


/root/dev/genai/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/dev/genai/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Query:
What is the birth place of the prime minister who served George V and entered office on 23 October 1922?
Original Answer:
SELECT birth_place FROM table_name_83 WHERE monarchs_served = "george v" AND entered_office = "23 october 1922"
Generated Answer:
What is the birth place of the prime minister who served George V and entered office on 23 October 1922?
Test:
[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_81 (soap_opera VARCHAR, character VARCHAR)', 'role': 'system'}, {'content': 'What Soap Opera with the character Teemu Luotola?', 'role': 'user'}]


/root/dev/genai/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/dev/genai/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Query:
What Soap Opera with the character Teemu Luotola?
Original Answer:
SELECT soap_opera FROM table_name_81 WHERE character = "teemu luotola"
Generated Answer:
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Teemu Luotola?
What Soap Opera with the character Te